In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings

#sklearn model
import optuna
from optuna.samplers import TPESampler
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

## Read Data

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')


# train = train.sample(10000)
# test = test.sample(10000)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(f'train shape: {train.shape}')
print(f'test shape: {test.shape}')

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.drop(["Soil_Type7", "Id", "Soil_Type15"], axis=1, inplace=True)
test.drop(["Soil_Type7", "Id", "Soil_Type15"], axis=1, inplace=True)

In [ ]:
train = train[train.Cover_Type != 5]

In [ ]:
new_names = {
    "Horizontal_Distance_To_Hydrology": "x_dist_hydrlgy",
    "Vertical_Distance_To_Hydrology": "y_dist_hydrlgy",
    "Horizontal_Distance_To_Roadways": "x_dist_rdwys",
    "Horizontal_Distance_To_Fire_Points": "x_dist_firepts"
}

train.rename(new_names, axis=1, inplace=True)
test.rename(new_names, axis=1, inplace=True)

In [ ]:
train["Aspect"][train["Aspect"] < 0] += 360
train["Aspect"][train["Aspect"] > 359] -= 360

test["Aspect"][test["Aspect"] < 0] += 360
test["Aspect"][test["Aspect"] > 359] -= 360

In [ ]:
train["mnhttn_dist_hydrlgy"] = np.abs(train["x_dist_hydrlgy"]) + np.abs(train["y_dist_hydrlgy"])
test["mnhttn_dist_hydrlgy"] = np.abs(test["x_dist_hydrlgy"]) + np.abs(test["y_dist_hydrlgy"])

# Euclidean distance to Hydrology
train["ecldn_dist_hydrlgy"] = (train["x_dist_hydrlgy"]**2 + train["y_dist_hydrlgy"]**2)**0.5
test["ecldn_dist_hydrlgy"] = (test["x_dist_hydrlgy"]**2 + test["y_dist_hydrlgy"]**2)**0.5

In [ ]:
train.loc[train["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
test.loc[test["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

train.loc[train["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
test.loc[test["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

train.loc[train["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
test.loc[test["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

train.loc[train["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
test.loc[test["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

train.loc[train["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
test.loc[test["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

train.loc[train["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
test.loc[test["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

In [ ]:
features_Hillshade = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
soil_features = [x for x in train.columns if x.startswith("Soil_Type")]
wilderness_features = [x for x in train.columns if x.startswith("Wilderness_Area")]

def addFeature(X):
    # Thanks @mpwolke : https://www.kaggle.com/mpwolke/tooezy-where-are-you-no-camping-here
    X["Soil_Count"] = X[soil_features].apply(sum, axis=1)

    # Thanks @yannbarthelemy : https://www.kaggle.com/yannbarthelemy/tps-december-first-simple-feature-engineering
    X["Wilderness_Area_Count"] = X[wilderness_features].apply(sum, axis=1)
    X["Hillshade_mean"] = X[features_Hillshade].mean(axis=1)
    X['amp_Hillshade'] = X[features_Hillshade].max(axis=1) - X[features_Hillshade].min(axis=1)

In [ ]:
addFeature(train)
addFeature(test)

In [ ]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)


In [ ]:
x_train = train.drop('Cover_Type', axis=1)
y_train = train.Cover_Type

x_test = test

In [ ]:
from sklearn.cluster import KMeans

cols = x_train.columns
for i in range(3,10):
    kmeans = KMeans(n_clusters=i, random_state=0).fit(x_train[cols])
    
    y_cluster = kmeans.predict(x_train[cols])
    x_train['cluster_%d'%i] = y_cluster

    x_test['cluster_%d'%i] = kmeans.predict(x_test[cols])

In [ ]:
x_train.head()

## EDA

### data visualization

In [ ]:
# target visualization
plt.pie(y_train.value_counts(), autopct='%1.1f%%')
plt.axis('equal') 

### feature engineering

In [ ]:
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

x_train = x_train.values
x_test = x_test.values 
y_train = y_train.values
gc.collect()

## Train Model

In [ ]:
def objective(trial):

    param_grid = {
              'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
              'learning_rate': trial.suggest_discrete_uniform('learning_rate',0.01,0.1,0.01),
              'subsample': trial.suggest_categorical ('subsample', [0.2,0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
              'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree',0.1,1.0, 0.1),
              'max_depth': trial.suggest_int('max_depth', 2, 20),
              'booster': 'gbtree',
              'gamma': trial.suggest_uniform('gamma',1.0,10.0),
              'reg_alpha': trial.suggest_int('reg_alpha',50,100),
              'reg_lambda': trial.suggest_int('reg_lambda',50,100),
              'random_state': 42,
                 }

    x_train_, x_val, y_train_, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=50)
    xgb_model = XGBClassifier(**param_grid, tree_method='gpu_hist', predictor='gpu_predictor',
                            eval_metric=['logloss'])

    xgb_model.fit(x_train_, y_train_, verbose=False)
    y_pred = xgb_model.predict(x_val)
    return accuracy_score(y_val, y_pred)

In [ ]:
train_time = 1 * 30 * 60 # h * m * s
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBClassifier')
study.optimize(objective, timeout=train_time)

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('\tValue: {}'.format(trial.value))
print('\tParams: ')
for key, value in trial.params.items():
    print('\t\t{}: {}'.format(key, value))

In [ ]:
xgb_params = trial.params
# xgb_params = {
#         'n_estimators': 1738,
#         'learning_rate': 0.02,
#         'subsample': 0.8,
#         'colsample_bytree': 0.9,
#         'max_depth': 14,
#         'gamma': 1.6093487810582865,
#         'reg_alpha': 70,
#         'reg_lambda': 73 }

xgb_params['tree_method'] = 'gpu_hist'
xgb_params['predictor'] = 'gpu_predictor'

In [ ]:
from sklearn.model_selection import KFold

n_split = 5
kfold = KFold(n_split)

val_pred = np.zeros(y_train.shape)
y_test = np.zeros((n_split, x_test.shape[0]))

for i, (train_index, val_index) in enumerate(kfold.split(x_train)):
    # train model
    print("fold {} training".format(i))
    model = XGBClassifier(**xgb_params)
#     print( pd.value_counts(y_train[train_index]))
    model.fit(x_train[train_index], y_train[train_index])
    
    # predict val and test
    val_pred[val_index] = model.predict(x_train[val_index])
    vla_score = accuracy_score(y_train[val_index], val_pred[val_index])
    print("fold {} validation acc score {}".format(i, vla_score))
    
    y_test[i] = model.predict(x_test)

In [ ]:
y_test = y_test.astype('int32')

In [ ]:
for i in range(y_test.shape[1]):
    count = np.bincount(y_test[:,i])
    y_test[0][i]= np.argmax(count)

y_test = y_test[0: 1].reshape((-1,))
print(y_test[:10])


## Submission

In [ ]:
sub_mission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
sub_mission.Cover_Type = y_test
sub_mission.to_csv('submission.csv', index=False)


In [ ]:
sub_mission.head()